Deutsch's Algorithm

In [133]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.visualization import plot_bloch_vector, plot_histogram
import matplotlib.pyplot as plt

In [ ]:
# Setting up registers and circuits
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

In [135]:
init = QuantumCircuit(qreg_q, creg_c)
init.x(qreg_q[1])
init.h(qreg_q[0])
init.h(qreg_q[1])
init.barrier()

# Simulate unbalanced function
unbalanced = QuantumCircuit(qreg_q, creg_c)
unbalanced.x(qreg_q[1])
unbalanced.barrier()

# Simulate balanced function
balanced = QuantumCircuit(qreg_q, creg_c)
# balanced.cx(qreg_q[0],1)
# balanced.barrier()

final = QuantumCircuit(qreg_q, creg_c)
final.h(qreg_q[0])
final.measure(qreg_q[0], 0)

tmp = init.compose(unbalanced, qreg_q)
circuit = tmp.compose(final, qreg_q)
print(circuit)

     ┌───┐      ░       ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘ ░       ║ 
c: 1/═══════════════════════════╩═
                                0 


In [136]:
# Simulating unbalanced:
result = AerSimulator().run(circuit, shots=100).result()
counts = result.get_counts()
print(counts)

{'0': 100}


In [137]:
# service = QiskitRuntimeService()

# backend = service.least_busy(simulator=False, operational=True)

# pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
# isa_circuit = pm.run(circuit)

# isa_circuit.draw("mpl", idle_wires=False)

Finalisation:

### structure:

# Intro

# Beginning
# Functions
# End

# Expected vs Real results

# Deutsch's Algorithm

Deutsch's Algorithm determines whether a function  
$$
f : \{0,1\} \rightarrow \{0,1\}
$$
is **constant** (always outputs the same value) or **balanced** (outputs 0 for one input and 1 for the other).  
Classically, this requires two evaluations, but the quantum version can decide with just one.



In [141]:
def deutsch_algo(oracle):
    qreg_q = QuantumRegister(2, 'q')
    creg_c = ClassicalRegister(1, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.x(qreg_q[1])
    circuit.h(qreg_q[0])
    circuit.h(qreg_q[1])
    circuit.barrier()

    circuit = circuit.compose(oracle)
    circuit.barrier()

    circuit.h(qreg_q[0])
    circuit.measure(qreg_q[0], 0)

    return circuit


## Implementing Constant and Balanced Functions

Constant functions can be represented by either doing nothing (always 0) or applying an X gate (always 1).

Balanced functions can be implemented in two common ways using controlled operations.

For simplicity, we will pick one example from each to demonstrate (x gate for constant, cx gate for balanced)

In [139]:
def unbalanced():
    unbalanced = QuantumCircuit(qreg_q, creg_c)
    unbalanced.x(qreg_q[1])
    return unbalanced

def balanced():
    balanced = QuantumCircuit(qreg_q, creg_c)
    balanced.cx(qreg_q[0],1)
    return balanced

## Mathematics for Constant and Balanced Functions
### Expected Outcomes

For constant functions, the result is always the same (all 0’s or all 1’s, ignoring noise).  
For balanced functions, the measurement outcomes should ideally be split evenly between 0 and 1.

In [140]:
for oracle in [unbalanced, balanced]:
    circuit = deutsch_algo(oracle())
    result = AerSimulator().run(circuit, shots=100).result()
    counts = result.get_counts()
    #print(counts)
    plot_histogram(counts)